## **Utilisation simple des identifieurs de 95 langues** (rev3) **avec les :**
>### **- '*Sparse*' Bag Of Words**
>### **- Tokenisations Tiktoken**
>### **- CountVectorizer utilisant une tokenisation '*custom*'**
>### **- Classificateur Naïve Bayesg**

#### **Choix du tokenizer** <font color='red'>(nécéssaire pour traduction texte libre)</font>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tiktoken
import random
import joblib
from sklearn.feature_extraction.text import CountVectorizer

# Ce parametre permet éventuellement d'équilibrer de nombre de phrase par langue.
# Si ce parametre est très grand, tout le corpus sera lu. 
nb_phrase_lang = 500000

import warnings
warnings.filterwarnings('ignore')

#### **Lectures des phrases de "sentences-big.csv", et de leur étiquette "Langue" pour les langues sélectionnées**

In [2]:
# Ouvrir le fichier d'entrée en mode lecture
def create_lang_df(path):
    df = pd.read_csv(path, index_col ='id')
    return df

df = create_lang_df('../data/multilingue/sentences-big.csv')
lan_code = list(set(df['lan_code']))
df_lan = pd.DataFrame(data= df.groupby('lan_code').size(), columns = ['nb_phrases_lang'] )

# Filtrage des langues qui ont peu de phrases (>2000)
df_lan = df_lan.loc[df_lan['nb_phrases_lang']>=2000]
list_lan = list(set(df_lan.index))

df = df.sample(frac=1, random_state=42).reset_index(drop=True)
n_rows = len(df)
print('Nombre de lignes de sentence.csv:',n_rows)
print('Nombre de langues à classer:',len(lan_code))
display(df.head(10))
display(df.tail(10))

Nombre de lignes de sentence.csv: 10341812
Nombre de langues à classer: 404


,lan_code,sentence
0,ita,Il tuo futuro è pieno di possibilità.
1,fra,"J'aimerais aller en France, un jour."
2,epo,La polica enketo aperigis ilian sekretan vivon.
3,kab,Kullec ifukk yid-k.
4,hun,Több munkát nem tudok elvállalni.
5,epo,Tiuj amikoj havas malbonan influon sur vi.
6,por,Se ao menos eu soubesse!
7,kab,"Kemm d yiwet seg timeddukal n Tom, neɣ ala?"
8,fra,Augmente le son.
9,hun,"Olyan keményen dolgoztam, amennyire csak lehet..."


,lan_code,sentence
10341802,hun,"Úgy gondolom, nagyon jó a munkád."
10341803,eng,We had our doubts.
10341804,ita,Va a imparare qualcosa in Irlanda?
10341805,mar,मी रांगेत शेवटी होते.
10341806,ber,Nṛuḥ ad d-naɣ imsured aɣurbiz.
10341807,deu,Wir werden das Problem nicht aufgreifen.
10341808,fra,Je suis cuit !
10341809,kab,Isefk fell-ak a tregleḍ iɣis-a.
10341810,tok,o pana ala e moku tawa soweli tomo.
10341811,hun,Máris unod magad?


#### **Préparation de la vectorisation par CountVectorizer** <font color='red'>(nécéssaire pour traduction texte libre)</font>
#### **Création de la fonction de création d'un Bags  Of Worlds pour les phrases à identifier** <font color='red'>(nécéssaire pour traduction texte libre)</font>

In [3]:
# Selection du tokenizer
tokenizer = tiktoken.get_encoding("cl100k_base")

# Les 2 fonctions suivantes sont nécéssaires afin de sérialiser ces parametre de CountVectorizer
# et ainsi de sauvegarder le vectorizer pour un un usage ultérieur sans utiliser X_train pour  le réinitialiser
def custom_tokenizer(text):
    tokens = tokenizer.encode(text)  # Cela divise le texte en mots
    return tokens

def custom_preprocessor(text):
    return text

# CountVectorizer a une liste de phrase en entrée.
# Cette fonction met les données d'entrée dans le bon format
def format_to_vectorize(data):
    X_tok = []
    if "DataFrame" in str(type(data)):sentences = df.tolist()
    elif "str" in str(type(data)):
        sentences =[data]
    else: sentences = data
                          
    for sentence in sentences:
        X_tok.append(sentence) # ('¤'.join([tokenizer.decode([ids]) for ids in tokenizer.encode(sentence)])+'¤')
    return X_tok

def create_BOW(data):
    global vectorizer
    
    X_tok = format_to_vectorize(data)
    X = vectorizer.transform(X_tok)
    return X

#### **Chargement du vectorizer** <font color='red'>(nécéssaire pour traduction texte libre)</font>

In [5]:
def load_vectorizer():
    global dict_token, dict_ids, nb_token
    
    path = '../data/vectorizer_tiktoken_big.pkl'
    vectorizer = joblib.load(path)
    dict_token = {tokenizer.decode([cle]): cle for cle, valeur in vectorizer.vocabulary_.items()}
    dict_ids = {cle: tokenizer.decode([cle]) for cle, valeur in vectorizer.vocabulary_.items()} #dict_ids.items()}
    nb_token = len(vectorizer.vocabulary_)
    return vectorizer

vectorizer = load_vectorizer()

#### **Chargement du classificateur entrainé avec l'algorithme Naïve Bayes** <font color='red'>(nécéssaire pour traduction texte libre)</font>

In [7]:
# from sklearn import naive_bayes

# Chargement du classificateur sauvé
clf_nb = joblib.load("../data/id_lang_tiktoken_nb_sparse_big.pkl")


#### **Definition de fonctions identificateur de langue avec Naive Bayes** <font color='red'>(nécéssaire pour traduction texte libre)</font>

In [10]:
import json

# Chargement du classificateur sauvegardé
clf_nb = joblib.load("../data/id_lang_tiktoken_nb_sparse_big.pkl")
vectorizer = load_vectorizer()

# Lisez le contenu du fichier JSON
with open('../data/multilingue/lan_to_language.json', 'r') as fichier:
    lan_to_language = json.load(fichier)



def lang_id_nb(sentences):
    if "str" in str(type(sentences)):
        return lan_to_language[clf_nb.predict(create_BOW(sentences))[0]]
    else: return [lan_to_language[l] for l in clf_nb.predict(create_BOW(sentences))]

In [11]:
def lang_id_gb(sentences):
    if "str" in str(type(sentences)):
        return lan_to_language[clf_gb.predict(create_BOW(sentences))[0]]
    else:
        return [lan_to_language[l] for l in clf_gb.predict(create_BOW(sentences))]        

#### **Exemples d'utilisation**

In [12]:
import random
# Instanciation d'exemples

sentence_no = random.sample(range(len(df)),5)

exemples = ["france is often snowy during spring , and it is relaxing in january .",
           "elle adore les voitures très luxueuses, et toi ?",
           "she loves very luxurious cars, don't you?",
           "vamos a la playa",
           "Ich heiße Keyne, und das ist wunderbar",
           "she loves you, mais elle te hait aussi, and das ist traurig", # Attention à cette phrase trilingue
           "I ate caviar", 
           "Vogliamo visitare il Colosseo e nuotare nel Tevere.",
            df['sentence'].iloc[sentence_no[0]],
            df['sentence'].iloc[sentence_no[1]],
            df['sentence'].iloc[sentence_no[2]],
            df['sentence'].iloc[sentence_no[3]],
            df['sentence'].iloc[sentence_no[4]],
          ]
lang_exemples = ['eng','fra','eng','spa','deu','e,f,d','en','ita',df['lan_code'].iloc[sentence_no[0]],df['lan_code'].iloc[sentence_no[1]],df['lan_code'].iloc[sentence_no[2]],
                 df['lan_code'].iloc[sentence_no[3]],df['lan_code'].iloc[sentence_no[4]]]
print('no\t lang\t Phrase')                            
for i in range(len(exemples)):
    print(i,'-\t',lang_exemples[i],'\t',exemples[i])

no	 lang	 Phrase
0 -	 eng 	 france is often snowy during spring , and it is relaxing in january .
1 -	 fra 	 elle adore les voitures très luxueuses, et toi ?
2 -	 eng 	 she loves very luxurious cars, don't you?
3 -	 spa 	 vamos a la playa
4 -	 deu 	 Ich heiße Keyne, und das ist wunderbar
5 -	 e,f,d 	 she loves you, mais elle te hait aussi, and das ist traurig
6 -	 en 	 I ate caviar
7 -	 ita 	 Vogliamo visitare il Colosseo e nuotare nel Tevere.
8 -	 fra 	 Il a dit qu'il ne connaissait pas cet homme, mais c'était un mensonge.
9 -	 jpn 	 私はお金を全部ではないにしても、新車を買うために３分の２以上は使ってしまった。
10 -	 deu 	 Man hört nur ihre Kritik!
11 -	 pol 	 Jutro oddam ci pieniądze, które mi pożyczyłeś.
12 -	 rus 	 Он предложил помочь.


In [17]:
# Affichage des prédictions
print("Langue réelle\tPréd. Naive B.\tPhrase")
for i in range(len(exemples)):
    print(lang_exemples[i]+'\t\t'+lang_id_nb(exemples[i])[:12]+'\t\t'+exemples[i])


Langue réelle	Préd. Naive B.	Phrase
eng		English		france is often snowy during spring , and it is relaxing in january .
fra		French		elle adore les voitures très luxueuses, et toi ?
eng		English		she loves very luxurious cars, don't you?
spa		Spanish		vamos a la playa
deu		German		Ich heiße Keyne, und das ist wunderbar
e,f,d		Galician		she loves you, mais elle te hait aussi, and das ist traurig
en		Turkish		I ate caviar
ita		Italian		Vogliamo visitare il Colosseo e nuotare nel Tevere.
fra		French		Il a dit qu'il ne connaissait pas cet homme, mais c'était un mensonge.
jpn		Japanese		私はお金を全部ではないにしても、新車を買うために３分の２以上は使ってしまった。
deu		German		Man hört nur ihre Kritik!
pol		Polish		Jutro oddam ci pieniądze, które mi pożyczyłeś.
rus		Russian		Он предложил помочь.


> **Recherche des phrases mal classées par Naive Bayes**

In [16]:
n_bad_max = 30
n_bad = 0
print("\tN°Ligne\tL. réelle\tPréd. Naive B.\t\tPhrase")
for i in range(len(df)):
    if (n_bad<n_bad_max):
        if (df['lan_code'].iloc[i] != lang_id_nb(df['sentence'].iloc[i])):
            n_bad +=1
            print(n_bad,'\t',i,'\t-',df['lan_code'].iloc[i],'\t\t'+lang_id_nb(df['sentence'].iloc[i]).ljust(12)[:12],'\t\t'+
                  df['sentence'].iloc[i]," (proba={:.2f}".format(max(clf_nb.predict_proba(create_BOW([df['sentence'].iloc[i]]))[0]))+")")

	N°Ligne	L. réelle	Préd. Naive B.		Phrase
1 	 0 	- ita 		Italian      		Il tuo futuro è pieno di possibilità.  (proba=1.00)
2 	 1 	- fra 		French       		J'aimerais aller en France, un jour.  (proba=1.00)
3 	 2 	- epo 		Esperanto    		La polica enketo aperigis ilian sekretan vivon.  (proba=1.00)
4 	 3 	- kab 		Berber langu 		Kullec ifukk yid-k.  (proba=0.71)
5 	 4 	- hun 		Hungarian    		Több munkát nem tudok elvállalni.  (proba=1.00)
6 	 5 	- epo 		Esperanto    		Tiuj amikoj havas malbonan influon sur vi.  (proba=1.00)
7 	 6 	- por 		Portuguese   		Se ao menos eu soubesse!  (proba=1.00)
8 	 7 	- kab 		Berber langu 		Kemm d yiwet seg timeddukal n Tom, neɣ ala?  (proba=1.00)
9 	 8 	- fra 		Interlingua  		Augmente le son.  (proba=0.67)
10 	 9 	- hun 		Hungarian    		Olyan keményen dolgoztam, amennyire csak lehetséges volt.  (proba=1.00)
11 	 10 	- eng 		English      		Has it been proven that there's a link between tobacco and lung cancer?  (proba=1.00)
12 	 11 	- fra 		French       		Vou